### ToDo:
1) Fine tune to specific data. <br/>

### Check to see if GPU enabled

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [1]:
#Download Coca dataset text corpus
#!wget https://raw.githubusercontent.com/mstatt/nlp-training/main/20-final-training-text-formatted.txt

In [2]:
import os
import joblib
import pickle
import time
import datetime
import torch
import torch.nn as nn
from torch.nn import functional as F

## Step1. Load Modules and Functions

In [3]:
trainingTextFile = '20-final-training-text-formatted.txt'
trainingTextFilefileName = trainingTextFile.rsplit('.', 1)[0]

## Step2. Set Model and Training Parameters

In [4]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 100000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 500
n_embd = 396
n_head = 6
n_layer = 10
dropout = 0.2
# ------------

## Step3. Load Model Class and Functions

In [5]:
torch.manual_seed(527)

with open(trainingTextFile, 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """
    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


## Step4. Create Bigram Model Class and Functions

In [6]:
# super simple bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


## Step5. Create and instance of the model 

In [7]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(round(sum(p.numel() for p in m.parameters())/1e6, 2), 'Million parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

19.03 Million parameters


### TRAIN A NEW LANGUAGE MODEL

Only necessary if you do not have a previously trasined model.<br/>
Time of start will be printed to the cell as well as time elapsed for each evaluation cycle.<br/>
End time will be printed as well.

In [ ]:
# RUN THIS CELL TO TRAIN A NEW MODEL

st = time.time()
x = datetime.datetime.now()
print("Start time: ",x.strftime("%c"))

for iter in range(max_iters):
    # Every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        et = time.time()
        elapsed_time = et - st
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
        # Save the model every 5k iterations
        if int(iter)%int(5000) == 0:
            pyTorchsavedModelName = str(time.strftime("%Y%m%d-%H%M%S"))+'-'+str(batch_size)+ '-'+str(block_size)+ '-'+str(iter)+ '-'+str(learning_rate)+ '-'+str(n_embd)+ '-'+str(n_head)+ '-'+str(n_layer)+".model"
            torch.save(model.state_dict(), pyTorchsavedModelName)

    # Sample a batch of data
    xb, yb = get_batch('train')
    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# -------------------------------------------------------------
# Display training time for the model based on iterations.
et = time.time()
elapsed_time = et - st
print()
print()
print()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)), '  for ',max_iters,' iterations')
x = datetime.datetime.now()
print("End time: ",x.strftime("%c"))
# -------------------------------------------------------------


Start time:  Sun Feb 12 12:38:44 2023
step 0: train loss 4.5310, val loss 4.5351
Execution time: 00:00:00
step 500: train loss 2.2306, val loss 2.2095
Execution time: 00:07:06
step 1000: train loss 1.7403, val loss 1.7002
Execution time: 00:14:07
step 1500: train loss 1.5617, val loss 1.5234
Execution time: 00:21:07
step 2000: train loss 1.4678, val loss 1.4358
Execution time: 00:28:08
step 2500: train loss 1.4141, val loss 1.3857
Execution time: 00:35:08
step 3000: train loss 1.3811, val loss 1.3550
Execution time: 00:42:09
step 3500: train loss 1.3464, val loss 1.3261
Execution time: 00:49:09
step 4000: train loss 1.3281, val loss 1.3096
Execution time: 00:56:09
step 4500: train loss 1.3067, val loss 1.2910
Execution time: 01:03:10
step 5000: train loss 1.2851, val loss 1.2702
Execution time: 01:10:10
step 5500: train loss 1.2702, val loss 1.2588
Execution time: 01:17:11
step 6000: train loss 1.2560, val loss 1.2460
Execution time: 01:24:11
step 6500: train loss 1.2423, val loss 1.23

## Save the Generatively Pretrained Transformer (GPT) model.
Save the model to use it yourself for inference: <br/> 
You save the model, you restore it, and then you change the model to evaluation mode. <br/> 
This is done because you usually have BatchNorm and Dropout layers that by default are in train mode on construction:

In [ ]:
# Explicitly save the model
pyTorchsavedModelName = str(time.strftime("%Y%m%d-%H%M%S"))+'-'+str(batch_size)+ '-'+str(block_size)+ '-'+str(max_iters)+ '-'+str(learning_rate)+ '-'+str(n_embd)+ '-'+str(n_head)+ '-'+str(n_layer)+".model"
torch.save(model.state_dict(), pyTorchsavedModelName)

## Generate text from the (GPT) model.

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

## Generate a output file of generated text from the (GPT) model.

In [ ]:
# Generate a output file of Model text.
# Output final Name
timestr = time.strftime("%Y%m%d-%H%M%S")
generatedTextFile = str(timestr+'.txt')
context = torch.zeros((1, 1), dtype=torch.long, device=device)
open(generatedTextFile, 'w', encoding='utf-8').write(decode(m.generate(context, max_new_tokens=5000)[0].tolist()))

## Restore a saved (GPT) model.

In [ ]:
modelToLoad = '20230212-123215-64-256-300-0.0003-396-6-10.model'
model.load_state_dict(torch.load(modelToLoad))
#model.eval()

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

## Save model to resume training later: <br/>
If you need to keep training the model that you are about to save, you need to save more than just the model. <br/>
You also need to save the state of the optimizer, epochs, score, etc. You would do it like this:

In [ ]:
#def savefortraining():
#    state = {
#    'epoch': epoch,
#    'state_dict': model.state_dict(),
#    'optimizer': optimizer.state_dict(),
#    }
#    torch.save(state, pyTorchsavedModelName)

To resume training you would do things like: 
state = torch.load(filepath), and then, <br/>
to restore the state of each individual object, something like this:

In [ ]:
# state = torch.load(filepath)
# model.load_state_dict(state['state_dict'])
# optimizer.load_state_dict(state['optimizer'])